In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 이 부분은 처음 한번만 실행하면 됌.
# 코드 수정 - "The reason is that the last Ubuntu update update supports chromium driver just via snap."
# 최근 우분투 업데이트에서 크롬 드라이버 설치를 snap을 이용해서만 하도록 바뀜
# 고로 snap 없이 설치하는 아래 우회 코드로 변경
# 출처 : https://colab.research.google.com/drive/1cbEvuZOhkouYLda3RqiwtbM-o9hxGLyC
# 출처2 : https://stackoverflow.com/questions/75155063/selenium-use-chrome-on-colab-got-unexpectedly-exited

%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium|
pip install selenium


Executing: /tmp/apt-key-gpghome.tAFVV6IvOh/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.NcK9Z1gehO/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.5e3FNXIs8y/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://d

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import time

import os
import progressbar
from more_itertools import chunked
import os
import pickle

In [ ]:
#Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.

options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/LiveCommerse/Total_DataFrame.csv')
df2 = df.sort_values(by='Num_Sub', ascending=False)
df3 = df2[df2['Tour/Experience'] == 1]
df4 = df3[(df3['Food']==0) & (df3['Beauty']==0) & (df3['Life']==0) & (df3['Fashion']==0) & (df3['Kids']==0) & (df3['Tech']==0) & (df3['Hobby/Leisure']==0) & (df3['Culture']==0)]
df5 = df3[(df3['Food']==1) | (df3['Beauty']==1) | (df3['Life']==1) | (df3['Fashion']==1) | (df3['Kids']==1) | (df3['Tech']==1) | (df3['Hobby/Leisure']==1) | (df3['Culture']==1)]
df5.Num_Video = df5['Num_Video'].astype(int)
nontour_df = df5[['Channel_Name', 'Channel_number', 'Num_Video']]

<ipython-input-6-073505ca664d>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/LiveCommerse/Total_DataFrame.csv')
<ipython-input-6-073505ca664d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5.Num_Video = df5['Num_Video'].astype(int)


In [ ]:
def Single_Crawler(url, path):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')        # Head-less 설정
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome('chromedriver', options=options)
    
    driver.get(url)
    save_number = str(url)
    save_number = save_number[44:50]
    time.sleep(10)

    buttun = driver.find_element(By.XPATH, '//*[@id="product-root"]/div/div/div/div/div/div[3]/div[2]/div[2]/div/div[3]/div[2]/button[1]')
    buttun.click()
    time.sleep(1)
    buttun.click()
    
    repeat_n = 14
    empty = []
    df = pd.DataFrame({'name' : empty, 'chat' : empty})

    for re_n in range(repeat_n):
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        temp_list = soup.select('div.CommentList_animation_area_Ca5J9')
    
        chat_list_temp = []
        for i in temp_list:
            for j in i:
                for k in j:
                    chat = k.select('span')
                    chat = str(chat)
                    chat_list_temp.append(chat)

        name_list_temp = []
        for i in temp_list:
            for j in i:
                name = j.select('strong')
                name = str(name)
                if 'nickname' in name:
                    name_list_temp.append(name)

        chat_list = []
        for i in chat_list_temp:
            try:
                i = (i.split('<')[1])
                i = (i.split('>')[1])
                chat_list.append(i)
            except:
                continue
            
        name_list = []
        for i in name_list_temp:
            try:
                i = (i.split('<')[1])
                i = (i.split('>')[1])
                name_list.append(i)
            except:
                continue
            
        df_temp = pd.DataFrame({ 'name' : name_list, 'chat' : chat_list})
        df = pd.concat([df, df_temp], ignore_index=True)
        df.drop_duplicates(inplace=True)
        df.reset_index(drop=True, inplace=True)
        df.to_csv(path + '/Video_{}.csv' .format(save_number), index=False)
        time.sleep(200)
        
    df.drop_duplicates(inplace=True)
    df.reset_index(drop=True, inplace=True)
    df.to_csv(path + '/Video_{}.csv' .format(save_number), index=False)
    
    driver.quit()

In [ ]:
with open('/content/drive/MyDrive/LiveCommerse/FUCKUGOOGLE/nonTour_df/Complete_list.pkl', 'rb') as f:
    Complete_list = pickle.load(f)
    
print(Complete_list)
print(len(Complete_list))
# del Complete_list[1]
# del Complete_list[1]

[85453, 19279, 91205, 157, 64400, 15871, 74158, 45281, 18394, 74465, 60012, 520, 65838, 67806, 80861, 74572, 60896, 61220, 764, 53725, 66218, 67896, 41453, 78630, 74437, 78294, 68551, 26875]
28


In [ ]:
path = '/content/drive/MyDrive/LiveCommerse/FUCKUGOOGLE/nonTour_df/'
Complete_list = []
check_list = []
    
for i in range(len(nontour_df)):
    temp = nontour_df.iloc[i]
    temp_id = temp[0]
    temp_number = temp[1]
    dir_path = path + f'{temp_id}_{temp_number}'
    
    with open('/content/drive/MyDrive/LiveCommerse/FUCKUGOOGLE/nonTour_df/Complete_list.pkl', 'rb') as f:
        Complete_list = pickle.load(f)
    
    if temp_number not in Complete_list:
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')        # Head-less 설정
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        driver = webdriver.Chrome('chromedriver', options=options)
        url = f'https://shoppinglive.naver.com/categories/dc:5?ownerIds={temp_number}'
        driver.get(url)
        time.sleep(3)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        url_list = soup.select('#content > section.TitleMoreLinkSection_wrap_mp\+Z0.TitleMoreLinkSection_has_last_child_padding_GHBry.TitleMoreLinkSection_has_next_border_z6TVX.TitleMoreLinkSection_tablet_k1mYW > div.VerticalCardList_wrap_responsive_F9HDD.VerticalCardList_tablet_d1tlh.VerticalCardList_is_infinite_V9p37 > div')
        temp_list = []
        
        for j in url_list:
            j = j.select('a')
            j = str(j)
            j = j.split('href="')[1]
            j = j.split('"><div')[0]
            temp_list.append(j)
            
        if len(temp_list) == 1:
            a = len(temp_list)
            print(f'Video_list: {temp_id}_{temp_number}_{a}, Crwaling Start')
            
            try:
                os.mkdir(dir_path)
            except:
                continue
        
            try:
                print('Processing ...')
                Single_Crawler(temp_list[0], dir_path)
                Complete_list.append(temp_number)
                with open('/content/drive/MyDrive/LiveCommerse/FUCKUGOOGLE/nonTour_df/Complete_list.pkl', 'wb') as f:
                    pickle.dump(Complete_list, f)
                print(f'Channel {temp_id}_{temp_number} Crawling End.')

            except:
                print(f'Fail. Channel = {temp_id}, Number = {temp_number}')
                print(temp_list)
                
            
        else:
            print(f'Check: {temp_id}_{temp_number}_Crawling FAIL')
            check_list.append(temp_number)
            with open('/content/drive/MyDrive/LiveCommerse/FUCKUGOOGLE/nonTour_df/Check_list.pkl', 'wb') as f:
                pickle.dump(check_list, f)
                

Check: 신세계라이브쇼핑_59307_Crawling FAIL
Check: dseason_69886_Crawling FAIL
Video_list: 케라스타즈_60754_1, Crwaling Start
Check: 주식회사 케이티알파_69485_Crawling FAIL
Check: 어바웃펫 aboutPet_13312_Crawling FAIL
Video_list: 핏펫_39206_1, Crwaling Start
Check: AK MALL_10971_Crawling FAIL
Check: 오라방_64059_Crawling FAIL
Check: 비발디파크_62214_Crawling FAIL
Check: 어바웃펫 aboutPet_13313_Crawling FAIL
Check: 다이아 샵라이브_59802_Crawling FAIL
Check: 경기도 사회적경제 쇼핑몰_48640_Crawling FAIL
Check: 핏펫_39207_Crawling FAIL
Check: 웨어고_86600_Crawling FAIL
Check: 요핑_63861_Crawling FAIL
Check: 라라스테이션_59983_Crawling FAIL
Check: 티나 스토어_65432_Crawling FAIL
Check: W-LIVE STUDIO_67881_Crawling FAIL
Check: 스타맵_588_Crawling FAIL
Check: 서울드래곤시티 THE 26_66254_Crawling FAIL
Check: BEART_66703_Crawling FAIL
Check: 디엘 스토어_29358_Crawling FAIL
Check: 이야기모바일_74490_Crawling FAIL
Check: 지유라이브_84005_Crawling FAIL
Check: 마켓윈썸_83695_Crawling FAIL
Check: 유진팡_35082_Crawling FAIL
Check: 광대라이브_66773_Crawling FAIL
Check: 마마프로젝트_88034_Crawling FAIL
Check: 대구관광라이브커머스